# 🩺 Clinical Assistant — PDF RAG (Colab v4.0, **T4 GPU • Fast • Reliable**)


- Drive-first + HF cache on Drive  
- Recursive PDF ingest + optional CSVs  
- Batched GPU embeddings (no Chroma embedder)  
- Subset mode for quick tests  
- Gradio UI with citations


Submitted By Team :- **Brocode **

In [1]:

!pip -q install torch --index-url https://download.pytorch.org/whl/cu118
!pip -q install sentence-transformers chromadb gradio pypdf pandas==2.2.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 24.4 MB/s et

In [3]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:

import os
os.environ["HF_HOME"] = "/content/drive/MyDrive/hf_cache"
os.environ["TRANSFORMERS_CACHE"] = "/content/drive/MyDrive/hf_cache"
os.environ["SENTENCE_TRANSFORMERS_HOME"] = "/content/drive/MyDrive/hf_cache"

BASE = "/content/drive/MyDrive"
PERSIST_DIR = f"{BASE}/clinical_store"
PDF_DIR     = f"{BASE}/clinical_pdfs"
CSV_DIR     = f"{BASE}/clinical_csvs"

for d in (PERSIST_DIR, PDF_DIR, CSV_DIR, os.environ["HF_HOME"]):
    os.makedirs(d, exist_ok=True)

print("PDF folder:", PDF_DIR)
print("CSV folder:", CSV_DIR)
print("Vector DB:", PERSIST_DIR)
print("HF cache:", os.environ["HF_HOME"])


PDF folder: /content/drive/MyDrive/clinical_pdfs
CSV folder: /content/drive/MyDrive/clinical_csvs
Vector DB: /content/drive/MyDrive/clinical_store
HF cache: /content/drive/MyDrive/hf_cache


In [5]:

import os, re, time, hashlib, textwrap
import pandas as pd
import gradio as gr
from pypdf import PdfReader
import chromadb
import torch
from sentence_transformers import SentenceTransformer

pd.set_option("display.max_colwidth", 240)

def sha1(s: str) -> str:
    import hashlib as _h
    return _h.sha1(s.encode("utf-8")).hexdigest()

def chunk_text(text, size=1800, overlap=50):
    toks = text.split()
    out = []
    i = 0
    while i < len(toks):
        out.append(" ".join(toks[i:i+size]))
        i += max(1, size - overlap)
    return out

def read_pdfs_recursive(root_dir: str):
    pages = []
    total_files = total_pages = 0
    start = time.time()
    for dirpath, _, files in os.walk(root_dir):
        for fname in files:
            if not fname.lower().endswith(".pdf"):
                continue
            total_files += 1
            path = os.path.join(dirpath, fname)
            try:
                reader = PdfReader(path)
                for i, page in enumerate(reader.pages):
                    try:
                        text = page.extract_text() or ""
                    except Exception:
                        continue
                    text = re.sub(r"\s+", " ", text).strip()
                    if text:
                        pages.append({"source": fname, "page": i+1, "text": text})
                        total_pages += 1
                        if total_pages % 200 == 0:
                            print(f"Read pages: {total_pages} (files: {total_files})")
            except Exception as e:
                print("PDF read error:", fname, e)
    print(f"Finished PDFs — files: {total_files}, pages: {total_pages}, time: {time.time()-start:.1f}s")
    return pages

def read_csvs(csv_dir: str):
    csvs = [p for p in os.listdir(csv_dir) if p.lower().endswith('.csv')]
    rows = []
    for fname in csvs:
        path = os.path.join(csv_dir, fname)
        try:
            df = pd.read_csv(path)
            for i, row in df.iterrows():
                parts = [f"{col}: {str(val).strip()}" for col, val in row.items() if pd.notna(val)]
                text = " | ".join(parts)
                rows.append({"source": fname, "row": int(i), "text": text})
        except Exception as e:
            print("CSV read error:", fname, e)
    return rows

USE_SUBSET = False
TARGET_FOLDERS = {"Covid", "Heart_attack"}

def read_pdfs_subset(root_dir: str, targets):
    pages = []
    for dirpath, _, files in os.walk(root_dir):
        parts = set(dirpath.split(os.sep))
        if not (parts & targets):
            continue
        for fname in files:
            if not fname.lower().endswith(".pdf"):
                continue
            path = os.path.join(dirpath, fname)
            try:
                reader = PdfReader(path)
                for i, page in enumerate(reader.pages):
                    try:
                        text = page.extract_text() or ""
                    except Exception:
                        continue
                    text = re.sub(r"\s+", " ", text).strip()
                    if text:
                        pages.append({"source": fname, "page": i+1, "text": text})
            except Exception as e:
                print("PDF read error:", fname, e)
    return pages


/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [6]:

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Embedding device:", device)
st_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)
ENC_BATCH = 128

client = chromadb.PersistentClient(path=PERSIST_DIR)
# Clean rebuild toggle
CLEAN_REBUILD = False
if CLEAN_REBUILD:
    try:
        client.delete_collection("clinical_pdf_store")
        print("Deleted old collection")
    except Exception:
        pass
try:
    collection = client.get_collection("clinical_pdf_store")
    print("Collection opened:", collection.name)
except Exception:
    collection = client.create_collection(name="clinical_pdf_store")
    print("Collection created:", collection.name)


Embedding device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Collection opened: clinical_pdf_store


In [7]:

start = time.time()

pdf_pages = read_pdfs_subset(PDF_DIR, TARGET_FOLDERS) if USE_SUBSET else read_pdfs_recursive(PDF_DIR)
csv_rows = read_csvs(CSV_DIR)

ids, docs, metas = [], [], []
chunk_count = 0

def push_chunks(uid_prefix, meta_obj, text):
    global chunk_count
    for ch in chunk_text(text):
        uid = f"{uid_prefix}::h{sha1(ch)[:12]}"
        ids.append(uid); docs.append(ch); metas.append(meta_obj)
        chunk_count += 1
        if chunk_count % 500 == 0:
            print(f"Prepared chunks: {chunk_count}")

for p in pdf_pages:
    push_chunks(f"pdf::{p['source']}::p{p['page']}", {"source": p["source"], "page": p["page"], "type":"pdf"}, p["text"])
for r in csv_rows:
    push_chunks(f"csv::{r['source']}::row{r['row']}", {"source": r["source"], "row": r["row"], "type":"csv"}, r["text"])

seen = set(); I, D, M = [], [], []
for i, d, m in zip(ids, docs, metas):
    if i in seen:
        continue
    seen.add(i); I.append(i); D.append(d); M.append(m)

print(f"Unique chunks to add: {len(I)}")

BATCH_ADD = 2000
added = 0; t0 = time.time(); total = len(I)

for s in range(0, total, BATCH_ADD):
    e = min(s + BATCH_ADD, total)
    batch_ids, batch_docs, batch_metas = I[s:e], D[s:e], M[s:e]

    batch_embs = st_model.encode(
        batch_docs, device=device, batch_size=ENC_BATCH,
        show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True
    ).tolist()

    collection.add(ids=batch_ids, embeddings=batch_embs, metadatas=batch_metas, documents=batch_docs)

    added += len(batch_ids)
    pct = 100.0 * added / total
    print(f"[{added}/{total}] {pct:5.1f}%  elapsed: {time.time()-t0:,.1f}s")

print(f"Indexed chunks this run: {added}")
print('Unique PDFs:', len({p['source'] for p in pdf_pages}))
print('CSVs:', sorted({r['source'] for r in csv_rows}))
print(f'Total time: {time.time()-start:,.1f}s')


Finished PDFs — files: 20, pages: 129, time: 18.4s
Prepared chunks: 500
Prepared chunks: 1000
Prepared chunks: 1500
Prepared chunks: 2000
Prepared chunks: 2500
Prepared chunks: 3000
Prepared chunks: 3500
Prepared chunks: 4000
Prepared chunks: 4500
Prepared chunks: 5000
Prepared chunks: 5500
Prepared chunks: 6000
Prepared chunks: 6500
Prepared chunks: 7000
Prepared chunks: 7500
Prepared chunks: 8000
Prepared chunks: 8500
Prepared chunks: 9000
Prepared chunks: 9500
Prepared chunks: 10000
Prepared chunks: 10500
Prepared chunks: 11000
Prepared chunks: 11500
Prepared chunks: 12000
Prepared chunks: 12500
Prepared chunks: 13000
Prepared chunks: 13500
Prepared chunks: 14000
Prepared chunks: 14500
Prepared chunks: 15000
Prepared chunks: 15500
Prepared chunks: 16000
Prepared chunks: 16500
Prepared chunks: 17000
Prepared chunks: 17500
Prepared chunks: 18000
Prepared chunks: 18500
Prepared chunks: 19000
Prepared chunks: 19500
Prepared chunks: 20000
Prepared chunks: 20500
Prepared chunks: 21000
Pre

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

[2000/34082]   5.9%  elapsed: 25.0s


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

[4000/34082]  11.7%  elapsed: 60.0s


...

[32000/34082]  93.9%  elapsed: 788.9s


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

[34000/34082]  99.8%  elapsed: 836.3s


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[34082/34082] 100.0%  elapsed: 839.5s
Indexed chunks this run: 34082
Unique PDFs: 20
CSVs: ['ctg-studies_Hearattack.csv', 'ctg-studies_KneeInjuries.csv', 'ctg-studies_covid.csv', 'ctg-studies_diabetes.csv']
Total time: 866.6s


In [8]:

def format_sources(metas, max_src=8):
    seen = set(); out = []
    for m in metas:
        key = (m.get("source"), m.get("page"), m.get("row"))
        if key in seen:
            continue
        seen.add(key)
        if m.get("type") == "pdf":
            out.append(f"{m['source']} p. {m.get('page')}")
        else:
            out.append(f"{m['source']} row {m.get('row')}")
        if len(out) >= max_src: break
    return out

def answer(query, k=6, max_chars=1500):
    if not query.strip():
        return "Please type a question.", []
    q_emb = st_model.encode([query], device=device, batch_size=1,
                            show_progress_bar=False, convert_to_numpy=True, normalize_embeddings=True).tolist()
    res = collection.query(query_embeddings=q_emb, n_results=int(k))
    docs  = res.get("documents", [[]])[0] or []
    metas = res.get("metadatas", [[]])[0] or []
    if not docs:
        return "(No relevant passages found in your local PDFs/CSVs.)", []
    pieces = [textwrap.shorten(d.replace("\n"," "), width=max_chars, placeholder="…") for d in docs[:3]]
    header = "Answer grounded **only** in your Drive PDFs/CSVs. "
    return header + "\n\n— ".join(pieces), format_sources(metas)


In [9]:

with gr.Blocks(title="Clinical Assistant — PDF RAG (GPU)") as demo:
    gr.Markdown("# 🩺 Clinical Assistant — **PDF RAG (GPU)**")
    gr.Markdown("Embeddings run on GPU. Answers cite filename + page from your PDFs/CSVs.")
    with gr.Row():
        q = gr.Textbox(label="Ask a clinical question", placeholder="e.g., Which models achieved highest F1 for COVID detection?")
        k = gr.Slider(3, 12, value=6, step=1, label="Top‑k passages")
    ask = gr.Button("Answer")
    ans = gr.Textbox(label="Grounded Answer", lines=12)
    src = gr.JSON(label="Sources (file & page)")

    def _go(query, topk):
        a, s = answer(query, k=topk)
        return a, s

    ask.click(_go, inputs=[q, k], outputs=[ans, src])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


<IPython.core.display.Javascript object>